# Main Analysis

This file does not rely on user input. So we would like to keep it interchangeable with the actual main_analysis.py file.
So please keep all the >relevant< code within one code block. (If you want to inspect certain elements or do other things that are not relevant to the main analysis, you can of course add them in new blocks).

In [1]:
import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import mne

from data_analysis.functions_preprocessing import \
    (combine_raws, split_raws, combine_epochs, split_epochs,
     preprocess_single_sub, load_ica, load_autoreject)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)
from data_analysis.functions_connectivity import \
    epochs_ispc
from data_analysis.functions_graph_theory import \
    epochs_weighted_small_world_coeff



subject_dir = "/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/"
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"



# Main Data Analysis ###################################
# initialize containers to analyze later
connectivity_matrices = []
small_world_coeffs = []

# Perform the data analysis
for subj_pair in ['202']:  #['202','203','204','205','206','207','208','209','211','212']:

    subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
    behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

    combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

    # split the subjects and delete the raw file
    raws = split_raws(combined_raw)
    del combined_raw
    
    for i, _ in enumerate(raws):
        # set the EEG Montage. We use 64 chans from the standard 10-05 system.
        montage = mne.channels.make_standard_montage("standard_1005")
        raws[i].set_montage(montage)

    # combine the subjects again
    raw_combined = combine_raws(raws[0], raws[1])
    del raws  # to save memory

    # do the behavioral analysis and get the epochs
    behavioral_df = calculate_alpha(pd.read_csv(behav_path))
    event_df = create_event_df(raw_combined)
    event_df = remove_ghost_triggers(event_df)
    event_df = join_event_dfs(event_df, behavioral_df)
    
    # get the first tap by looking at the first sample in each trial
    min_idx = event_df.groupby(["trial"])["sample"].idxmin()
    early_df = event_df[event_df.index.isin(min_idx)]
    early_events = events_from_event_df(early_df)
    
    # get the late taps by looking at the last sample - 1.5 seconds
    max_idx = event_df.groupby(["trial"])["sample"].idxmax()
    late_df = event_df[event_df.index.isin(max_idx)]
    late_events = events_from_event_df(late_df)
    late_events[:,0] -= int(raw_combined.info["sfreq"] * 1.5)
    
    # get the baseline events
    base_events = mne.pick_events(mne.find_events(raw_combined, shortest_event=1),
                              include=48)

    # define the parameters for epoching
    # TODO: Define events more elaborate!
    combined_events = np.vstack([base_events, early_events, late_events])
    tmin = 0
    tmax = 1.5

    # epoch the data. Here we filter out bad segments from both participants
    # TODO: do we need a baseline for the connectivity analysis?
    epochs = mne.Epochs(raw_combined, combined_events, tmin=tmin, tmax=tmax,
                        picks=["eeg"], baseline=None, preload=True) # only use the first two epochs
    
    # split the epochs to apply AR & ICA individually
    epochs_split = list(split_epochs(epochs))
    for i, cur_eps in enumerate(epochs_split):
        subj_id = "sub-{0}_p-{1}".format(subj_pair, i)

        # apply autoreject (exclude bads and interpolate)
        # TODO: The Autoreject guys apply ICA first and then autoreject local. i would do the same
        ar = load_autoreject(subj_id)
        cur_eps = ar.transform(cur_eps, return_log=False)

        # apply ICA
        ica = load_ica(subj_id)
        cur_eps = ica.apply(cur_eps)
        
        # rereference to avg ref
        cur_eps.set_eeg_reference(ref_channels='average')

        # APPLY LAPLACIAN
        cur_eps = mne.preprocessing.compute_current_source_density(cur_eps,
                                                                   stiffness=10,
                                                                   lambda2=1e-5)
        epochs_split[i] = cur_eps

    #combine the epochs again
    epochs = combine_epochs(epochs_split[0], epochs_split[1])
    del epochs_split
    
    # frequencies should be 25 freqs, log spaced between 4 and 50
    np.logspace(np.log10(4), np.log10(50), 25)
    cycles = freqs / 2.
    
    # calculate the ISPC
    ispc_matrix, freqs, times = epochs_ispc(epochs, freqs, cycles, n_jobs=4)
    
    print("ISPC DONE")
    
    # calculate the small world coefficient
    small_worlds = epochs_weighted_small_world_coeff(ispc_matrix)
    
    # append the results to the respective lists
    connectivity_matrices.append(ispc_matrix)
    small_world_coeffs.append(small_worlds)




Bad key "text.kerning_factor" on line 4 in
/net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif...


<ipython-input-1-da680b7874bf>:40: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3725311 =      0.000 ...  3637.999 secs
Ready.
Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg-1.fif...
Isotrak not found
    Range : 3725312 ... 5675445 =   3638.000 ...  5542.427 secs
Ready.
Reading 0 ... 5675445  =      0.000 ...  5542.427 secs...
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
6648 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
6648 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
900 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 90

<ipython-input-1-da680b7874bf>:84: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw_combined, combined_events, tmin=tmin, tmax=tmax,


0 bad epochs dropped



Dropped 418 epochs: 0, 1, 2, 3, 4, 5, 8, 9, 10, 12, 16, 17, 22, 23, 24, 28, 30, 31, 35, 36, 39, 40, 41, 47, 49, 52, 53, 59, 60, 61, 64, 65, 66, 67, 70, 71, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 99, 100, 102, 114, 116, 120, 121, 123, 132, 133, 135, 138, 140, 144, 145, 146, 147, 152, 155, 158, 160, 166, 167, 168, 170, 173, 174, 175, 176, 178, 179, 180, 181, 182, 184, 185, 186, 188, 190, 191, 192, 193, 197, 199, 203, 206, 208, 210, 211, 212, 213, 215, 217, 218, 219, 220, 222, 223, 224, 225, 227, 229, 231, 232, 235, 237, 238, 240, 242, 246, 250, 251, 252, 254, 255, 256, 257, 258, 259, 260, 265, 267, 270, 275, 277, 279, 282, 283, 284, 285, 286, 288, 290, 291, 293, 295, 297, 299, 312, 313, 316, 318, 322, 324, 325, 330, 331, 336, 338, 341, 345, 347, 354, 356, 358, 360, 361, 362, 364, 373, 381, 383, 386, 389, 390, 391, 392, 393, 394, 395, 398, 403, 404, 407, 409, 413, 425, 430, 434, 435, 440, 442, 443, 444, 446, 457, 461, 464, 467, 470, 471, 474, 478, 485, 48


Dropped 46 epochs: 2, 23, 44, 48, 50, 52, 53, 55, 67, 68, 79, 83, 90, 107, 126, 145, 146, 157, 164, 174, 181, 185, 195, 197, 199, 211, 226, 239, 242, 254, 256, 273, 274, 275, 276, 277, 284, 293, 320, 524, 582, 673, 682, 702, 738, 765
Reading /net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/bads/bad_components/sub-202_p-1-ica.fif ...
Now restoring ICA solution ...
Ready.
Transforming to ICA space (63 components)
Zeroing out 2 ICA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Fitted sphere radius:         95.1 mm
Origin head coordinates:      0.9 6.5 47.0 mm
Origin device coordinates:    0.9 6.5 47.0 mm


AssertionError: All data dimensions except channels must match, got [ 482 1537] != [ 854 1537]

In [40]:
# split the epochs to apply AR & ICA individually
epochs_split = list(split_epochs(epochs))
for i, cur_eps in enumerate(epochs_split):
    subj_id = "sub-{0}_p-{1}".format(subj_pair, i)
    
    # apply ICA
    ica = load_ica(subj_id)
    cur_eps = ica.apply(cur_eps)

    # apply autoreject (exclude bads and interpolate)
    # TODO: The Autoreject guys apply ICA first and then autoreject local. i would do the same
    ar = load_autoreject(subj_id)
    cur_eps = ar.transform(cur_eps, return_log=False)



    # rereference to avg ref
    cur_eps.set_eeg_reference(ref_channels='average')

    # APPLY LAPLACIAN
    cur_eps = mne.preprocessing.compute_current_source_density(cur_eps,
                                                               stiffness=10,
                                                               lambda2=1e-5)
    epochs_split[i] = cur_eps

Reading /net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/bads/bad_components/sub-202_p-0-ica.fif ...
Now restoring ICA solution ...
Ready.
Transforming to ICA space (63 components)
Zeroing out 1 ICA component



Dropped 378 epochs: 0, 1, 3, 4, 5, 8, 9, 10, 12, 16, 19, 22, 23, 24, 28, 30, 31, 35, 36, 40, 41, 47, 49, 52, 53, 59, 60, 61, 64, 65, 66, 67, 70, 71, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 97, 99, 100, 102, 114, 116, 120, 121, 122, 129, 132, 133, 135, 138, 140, 144, 145, 146, 147, 152, 154, 155, 158, 160, 166, 167, 168, 170, 174, 175, 176, 178, 179, 180, 181, 182, 184, 185, 186, 188, 190, 191, 192, 193, 197, 199, 202, 203, 206, 208, 210, 211, 212, 213, 215, 217, 218, 219, 220, 222, 223, 224, 225, 227, 229, 231, 232, 235, 237, 238, 240, 242, 246, 250, 251, 252, 254, 255, 256, 258, 259, 260, 265, 267, 270, 275, 277, 279, 282, 283, 284, 285, 286, 288, 290, 291, 293, 295, 297, 299, 312, 315, 316, 318, 322, 324, 330, 331, 336, 341, 345, 346, 356, 358, 360, 361, 362, 364, 373, 381, 383, 390, 391, 392, 393, 394, 395, 398, 403, 404, 407, 409, 413, 430, 434, 435, 440, 442, 443, 444, 446, 457, 467, 470, 471, 474, 478, 484, 485, 486, 487, 489, 490, 491, 493, 494, 495, 496


Dropped 8 epochs: 44, 53, 90, 146, 157, 164, 211, 293
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Fitted sphere radius:         95.1 mm
Origin head coordinates:      0.9 6.5 47.0 mm
Origin device coordinates:    0.9 6.5 47.0 mm


In [42]:
a = [True if i ==[] else False for i in epochs_split[0].drop_log]
b = [True if i ==[] else False for i in epochs_split[1].drop_log]

drop_mask = np.logical_and(a, b)
drop_log = [a + b for a, b in zip(epochs_split[0].drop_log, epochs_split[1].drop_log)]
np.sum(drop_mask)



#np.array(epochs_split[0].drop_log)
#[i for i in epochs_split[0].drop_log if ]
#epochs_split[i].drop_log != []
#np.logical_and(, epochs_split[0].drop_log)

AttributeError: 'AutoReject' object has no attribute 'reject_log'

In [ ]:
# get number of bad_epochs
combined_epochs = np.logical_or(ars[0].get_reject_log(epochs).bad_epochs, ars[1].get_reject_log(epochs).bad_epochs)

In [ ]:
sum(combined_epochs)

In [ ]:
subj_id